In [1]:
import numpy as np
import cirq
from clock import *

### AA function

In [190]:

def permute_qubits(matrix: np.ndarray, perm: list) -> np.ndarray:
    """
    Permute the qubit ordering of a 2^n x 2^n matrix.

    Args:
        matrix: The input operator matrix of shape (2^n, 2^n).
        perm: A list of length n specifying the new ordering of the qubits.
              For example, perm=[2,0,1,3] means:
                qubit 0 -> position 2,
                qubit 1 -> position 0,
                qubit 2 -> position 1,
                qubit 3 -> position 3.

    Returns:
        The permuted matrix of the same shape.
    """
    n = int(np.log2(matrix.shape[0]))
    assert matrix.shape == (2**n, 2**n), "Matrix must be 2^n x 2^n"
    assert sorted(perm) == list(range(n)), "perm must be a permutation of [0..n-1]"

    # Reshape into a tensor with separate row and column qubit axes
    tensor = matrix.reshape([2]*n + [2]*n)

    # Build transpose axes: first the row-qubit axes in new order, then the col-qubit axes
    axes = perm + [p + n for p in perm]

    # Transpose and collapse back to 2^n x 2^n
    permuted = tensor.transpose(axes).reshape(2**n, 2**n)
    return permuted

def reflector_1(n_qubits,index):
    label_qubits = len(index)
    qubit_indices = list(range(n_qubits))
    for i in index:
        qubit_indices.remove(i)
    qubit_indices = qubit_indices[::-1] + index
    #qubit_indices = [4,2,1,0,3]
    print(qubit_indices)
    ref_label = np.eye(2**label_qubits)
    ref_label[0][0] = -1
    ref_label_full = np.kron(np.eye(2**(n_qubits-label_qubits)),ref_label)
    ref_label_full = permute_qubits(ref_label_full, qubit_indices)
    return ref_label_full

def aa(index,U_prep,repetition):
    n_qubits = int(np.log2(U_prep.shape[0]))
    aa_circuit = np.eye(2**n_qubits)
    state = np.zeros(2**n_qubits)
    state[0] = 1
    prep_state = U_prep@state
    proj = np.outer(prep_state,prep_state.conj())
    reflector = np.eye(proj.shape[0]) - 2*proj
    print("Unitarity test for reflector ", -np.allclose(reflector@reflector.T.conj(), np.eye(reflector.shape[0])))
    for i in range(repetition):
        aa_circuit = -reflector@reflector_1(n_qubits,index)@aa_circuit
        print("Unitarity test for reflector ", np.allclose(reflector_1(n_qubits,index)@reflector_1(n_qubits,index).T.conj(), np.eye(reflector_1(n_qubits,index).shape[0])))
    return aa_circuit



## Testbed

In [232]:
g = -0.5
n_qubits = 6
post_selection_list = [0,n_qubits+1]
new_state,unitaries_list= sequential_prep_unitary(g,n_qubits)
U = np.array(product_unitaries_loop(unitaries_list[::-1]))
Uprime = aa(post_selection_list,U,1)@np.array(product_unitaries_loop(unitaries_list[::-1]))

Unitarity test for reflector  -1
[6, 5, 4, 3, 2, 1, 0, 7]
[6, 5, 4, 3, 2, 1, 0, 7]
[6, 5, 4, 3, 2, 1, 0, 7]
[6, 5, 4, 3, 2, 1, 0, 7]
Unitarity test for reflector  True


In [233]:
zero_state = np.zeros(U.shape[0])
zero_state[0] = 1
final_state = U@zero_state
Uprime_state = Uprime@zero_state

In [234]:
measure_qubits_outcome(final_state,post_selection_list,[0,0])

(array([ 0.29609328+0.j, -0.14804664+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j,  0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j,
        -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j,  0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j,
         0.07402332+0.j, -0.03701166+0.j,  0.07402332+0.j,  0.07402332+0.j,
        -0.14804664+0.j,  0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j,
        -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j, -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j,
         0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j, -0.14804664+0.j,
         0.07402332+0.j,  0.07402332+0.j, -0.03701166+0.j,  0.07402332+0.j,
         0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j, -0.14804664+0.j,
        -0.14804664+0.j, -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j,
         0.0

In [235]:
measure_qubits_outcome(Uprime_state,post_selection_list,[0,0])

(array([ 0.29609328+0.j, -0.14804664+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j,  0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j,
        -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j,  0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j,
         0.07402332+0.j, -0.03701166+0.j,  0.07402332+0.j,  0.07402332+0.j,
        -0.14804664+0.j,  0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j,
        -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j, -0.14804664+0.j,
        -0.14804664+0.j, -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j,
         0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j, -0.14804664+0.j,
         0.07402332+0.j,  0.07402332+0.j, -0.03701166+0.j,  0.07402332+0.j,
         0.07402332+0.j,  0.07402332+0.j,  0.07402332+0.j, -0.14804664+0.j,
        -0.14804664+0.j, -0.14804664+0.j,  0.07402332+0.j, -0.14804664+0.j,
         0.0

# AA is included in the main function
